# Librerias fundamentales para manipular datos y herramienta de optimizacion

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# leer la hoja de excel

In [3]:
data = pd.read_excel("/content/Datos para Problema de Optimización.xlsx")

# Limpiar y separar las tablas que se van a utilizar en el modelo

In [4]:
demanda_de_cada_ciudad = data.iloc[1:8,1:7]
capacidad_de_produccion_de_cada_fabrica = data.iloc[12:18,1:7]
inventario_inicial = data.iloc[22:25,1:3]
costo_de_produccion = data.iloc[28:34,1:7]
costo_mensual_de_inventario = data.iloc[37:40,1:7]
costo_fijo_de_fabrica_tercerizada = data.iloc[43:47,1:7]
costo_de_transporte  = data.iloc[50:,1:]

demanda_de_cada_ciudad.columns = demanda_de_cada_ciudad.iloc[0]
demanda_de_cada_ciudad = demanda_de_cada_ciudad.iloc[1:]
d_cols = demanda_de_cada_ciudad.columns
demanda_de_cada_ciudad = demanda_de_cada_ciudad.set_index(d_cols[0])

capacidad_de_produccion_de_cada_fabrica.columns = capacidad_de_produccion_de_cada_fabrica.iloc[0]
capacidad_de_produccion_de_cada_fabrica = capacidad_de_produccion_de_cada_fabrica.iloc[1:]
d_cols = capacidad_de_produccion_de_cada_fabrica.columns
capacidad_de_produccion_de_cada_fabrica = capacidad_de_produccion_de_cada_fabrica.set_index(d_cols[0])

inventario_inicial.columns = inventario_inicial.iloc[0]
inventario_inicial = inventario_inicial.iloc[1:]
d_cols = inventario_inicial.columns
inventario_inicial = inventario_inicial.set_index(d_cols[0])

costo_de_produccion.columns = costo_de_produccion.iloc[0]
costo_de_produccion = costo_de_produccion.iloc[1:]
d_cols = costo_de_produccion.columns
costo_de_produccion = costo_de_produccion.set_index(d_cols[0])

costo_mensual_de_inventario.columns = costo_mensual_de_inventario.iloc[0]
costo_mensual_de_inventario = costo_mensual_de_inventario.iloc[1:]
d_cols = costo_mensual_de_inventario.columns
costo_mensual_de_inventario = costo_mensual_de_inventario.set_index(d_cols[0])

costo_fijo_de_fabrica_tercerizada.columns = costo_fijo_de_fabrica_tercerizada.iloc[0]
costo_fijo_de_fabrica_tercerizada = costo_fijo_de_fabrica_tercerizada.iloc[1:]
d_cols = costo_fijo_de_fabrica_tercerizada.columns
costo_fijo_de_fabrica_tercerizada = costo_fijo_de_fabrica_tercerizada.set_index(d_cols[0])

costo_de_transporte.columns = costo_de_transporte.iloc[0]
costo_de_transporte = costo_de_transporte.iloc[1:]
d_cols = costo_de_transporte.columns
costo_de_transporte = costo_de_transporte.set_index(d_cols[0])

# Transformacion de los datos 
Pasar las filas a columnas y viceversa de la tabla costo de transporte, añadirle columna de Lima Terceros, luego cambiar el nombre de sus columnas para añadirles si son propias o de terceros, condersarlas en tres columnas que almacenen toda la info de
 origen, destino y costos de transporte; para finalmente unirla con la tabla de costos de produccion y hallar un costo total


In [5]:
costo_de_transporte_T = costo_de_transporte.T
costo_de_transporte_T["Lima Terceros"] = costo_de_transporte_T["Lima"]
costo_de_transporte_T = costo_de_transporte_T[sorted(costo_de_transporte_T.columns)]
costo_de_transporte_T.columns = costo_de_produccion.sort_index().index
costo_de_transporte_2 = costo_de_transporte_T.T.reset_index()
costo_de_transporte_2.columns = costo_de_transporte_2.columns.fillna("index")

costo_de_transporte_serializado = pd.melt(costo_de_transporte_2,id_vars=["index"],value_vars=costo_de_transporte_2.columns)
costo_de_transporte_serializado = costo_de_transporte_serializado.rename(columns={"index":"Desde",50:"Hacia","value":"costo de transporte"})

costo_de_transporte_serializado = costo_de_transporte_serializado.merge(costo_de_produccion.iloc[:,0],right_index=True,left_on="Desde",how="left")
costo_de_transporte_serializado = costo_de_transporte_serializado.rename(columns={costo_de_transporte_serializado.columns[-1]:"costo de produccion"})

costo_de_transporte_serializado["costo total de entrega"] = costo_de_transporte_serializado["costo de transporte"]+costo_de_transporte_serializado["costo de produccion"]
costo_de_transporte_serializado = costo_de_transporte_serializado.sort_values(["Desde","Hacia"])

In [6]:
def programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(mes:int):
    
    A_ub = np.full([5,30],0)
    b_ub = capacidad_de_produccion_de_cada_fabrica.sort_index().iloc[:,mes].values
    A_eq = np.full([6,30],0)

    for i in range(5):
        A_ub[i,6*i:6*i+6] =  1

    for i in range(6):
        A_eq[i,5*i:5*i+5] = 1

    A_eq = A_eq.T.reshape(6,30)
    b_eq = demanda_de_cada_ciudad_descontando_inventario.sort_index().iloc[:,mes].values
    c = costo_de_transporte_serializado["costo total de entrega"].values

    resultado = linprog(c=c,method="simplex",A_eq=A_eq,b_eq=b_eq,A_ub=A_ub,b_ub=b_ub,bounds=(0,None))

    return resultado

In [26]:

almacen_inicial_enero = np.array([4e+4,6e+4,0,0,0,0])

almacen_final_enero = np.array([0e+4,0e+4,0,0,0,0])
almacen_final_febrero = np.array([4e+4,6e+4,0,0,0,0])
almacen_final_marzo = np.array([0e+4,0e+4,0,0,0,0])
almacen_final_abril = np.array([5e+4,7.5e+4,0,0,0,0])
almacen_final_mayo = np.array([0e+4,0e+4,0,0,0,0])

almacen_inicial_febrero = almacen_final_enero
almacen_inicial_marzo = almacen_final_febrero
almacen_inicial_abril = almacen_final_marzo
almacen_inicial_mayo = almacen_final_abril


demanda_de_cada_ciudad_descontando_inventario = demanda_de_cada_ciudad.copy()

demanda_de_cada_ciudad_descontando_inventario.iloc[:,0] -= almacen_inicial_enero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,1] -= almacen_inicial_febrero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,2] -= almacen_inicial_marzo
demanda_de_cada_ciudad_descontando_inventario.iloc[:,3] -= almacen_inicial_abril
demanda_de_cada_ciudad_descontando_inventario.iloc[:,4] -= almacen_inicial_mayo

demanda_de_cada_ciudad_descontando_inventario.iloc[:,0] += almacen_final_enero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,1] += almacen_final_febrero
demanda_de_cada_ciudad_descontando_inventario.iloc[:,2] += almacen_final_marzo
demanda_de_cada_ciudad_descontando_inventario.iloc[:,3] += almacen_final_abril
demanda_de_cada_ciudad_descontando_inventario.iloc[:,4] += almacen_final_mayo

resultado_enero = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(0)
resultado_febrero = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(1)
resultado_marzo = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(2)
resultado_abril = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(3)
resultado_mayo = programacion_lineal_para_optimizar_los_costes_de_produccion_y_transporte(4)

In [27]:
resultado_mayo

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 765967.9700000001
 message: 'Optimization terminated successfully.'
     nit: 28
   slack: array([   0.,    0.,    0., 1487.,    0.])
  status: 0
 success: True
       x: array([222183., 127817.,      0.,      0.,      0.,      0.,      0.,
        22131., 147869.,      0.,      0.,      0.,      0.,  11939.,
            0., 801337., 155003., 231721.,      0., 198513.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,  80000.])

In [31]:
demanda_de_cada_ciudad_descontando_inventario-demanda_de_cada_ciudad

1,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01
NaT,,,,,
Lima,-40000.0,40000.0,-40000.0,50000.0,-50000.0
Arequipa,-60000.0,60000.0,-60000.0,75000.0,-75000.0
Cusco,0.0,0.0,0.0,0.0,0.0
Trujillo,0.0,0.0,0.0,0.0,0.0
Huancayo,0.0,0.0,0.0,0.0,0.0
Piura,0.0,0.0,0.0,0.0,0.0


In [28]:
demanda_de_cada_ciudad_descontando_inventario

1,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01
NaT,,,,,
Lima,689005.0,755964.0,735595.0,856822.0,801337.0
Arequipa,176002.0,269188.0,223185.0,352543.0,222183.0
Cusco,178700.0,198230.0,197243.0,257970.0,360400.0
Trujillo,244038.0,226396.0,290466.0,301522.0,311721.0
Huancayo,139726.0,117918.0,117809.0,142070.0,147869.0
Piura,90655.0,78431.0,117874.0,120044.0,155003.0


In [14]:
costo_de_transporte_serializado["produccion enero 2022"] = resultado_enero.x
costo_de_transporte_serializado["produccion febrero 2022"] = resultado_febrero.x
costo_de_transporte_serializado["produccion marzo 2022"] = resultado_marzo.x
costo_de_transporte_serializado["produccion abril 2022"] = resultado_abril.x
costo_de_transporte_serializado["produccion mayo 2022"] = resultado_mayo.x

In [33]:
tabla_de_resultados = pd.DataFrame(index=["Inventario inicial","Produccion","Envios","Inventario final"])

inventario_inicial_total_enero = almacen_inicial_enero.sum()
produccion_total_enero = costo_de_transporte_serializado.iloc[:,-5].sum()
inventario_final_total_enero = almacen_final_enero.sum()
envios_enero = inventario_inicial_total_enero+produccion_total_enero-inventario_final_total_enero

inventario_inicial_total_febrero = almacen_inicial_febrero.sum()
produccion_total_febrero = costo_de_transporte_serializado.iloc[:,-4].sum()
inventario_final_total_febrero = almacen_final_febrero.sum()
envios_febrero = inventario_inicial_total_febrero+produccion_total_febrero-inventario_final_total_febrero

inventario_inicial_total_marzo = almacen_inicial_marzo.sum()
produccion_total_marzo = costo_de_transporte_serializado.iloc[:,-3].sum()
inventario_final_total_marzo = almacen_final_marzo.sum()
envios_marzo = inventario_inicial_total_marzo+produccion_total_marzo-inventario_final_total_marzo

inventario_inicial_total_abril = almacen_inicial_abril.sum()
produccion_total_abril = costo_de_transporte_serializado.iloc[:,-2].sum()
inventario_final_total_abril = almacen_final_abril.sum()
envios_abril = inventario_inicial_total_abril+produccion_total_abril-inventario_final_total_abril

inventario_inicial_total_mayo = almacen_inicial_mayo.sum()
produccion_total_mayo = costo_de_transporte_serializado.iloc[:,-1].sum()
inventario_final_total_mayo = almacen_final_mayo.sum()
envios_mayo = inventario_inicial_total_mayo+produccion_total_mayo-inventario_final_total_mayo

tabla_de_resultados["Enero"] = [inventario_inicial_total_enero,produccion_total_enero,envios_enero,inventario_final_total_enero]
tabla_de_resultados["Febrero"] = [inventario_inicial_total_febrero,produccion_total_febrero,envios_febrero,inventario_final_total_febrero]
tabla_de_resultados["Marzo"] = [inventario_inicial_total_marzo,produccion_total_marzo,envios_marzo,inventario_final_total_marzo]
tabla_de_resultados["Abril"] = [inventario_inicial_total_abril,produccion_total_abril,envios_abril,inventario_final_total_abril]
tabla_de_resultados["Mayo"] = [inventario_inicial_total_mayo,produccion_total_mayo,envios_mayo,inventario_final_total_mayo]

tabla_de_resultados

,Enero,Febrero,Marzo,Abril,Mayo
Inventario inicial,100000.0,0.0,100000.0,0.0,125000.0
Produccion,1518126.0,1646127.0,1682172.0,1905971.0,2000000.0
Envios,1618126.0,1546127.0,1782172.0,1780971.0,2125000.0
Inventario final,0.0,100000.0,0.0,125000.0,0.0
